In [22]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling

# path to original file, template, and output folder
src_path      = "Annual_NLCD_LndCov_2000_CU_C1V1.tif"
template_path = "USA_1km_raster.tif"
dst_path      = "mrlc_pasture_2000_1km.tif"

# open template to get target grid
with rasterio.open(template_path) as tmpl:
    dst_height, dst_width = tmpl.height, tmpl.width
    dst_transform = tmpl.transform
    dst_crs = tmpl.crs
    dst_meta = tmpl.meta.copy()
    dst_meta.update(dtype="uint8", count=1)

# set up destination array
dst_data = np.zeros((dst_height, dst_width), dtype=np.uint8)

# open original file and reproject
with rasterio.open(src_path) as src:

    # reproject NLCD to 1 km using mode resampling
    # check if class==81
    tmp_1km = np.zeros((dst_height, dst_width), dtype=np.uint8)

    reproject(
        source=rasterio.band(src, 1),
        destination=tmp_1km,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.mode
    )

    # convert to binary mask: 1 = pasture/hay (class 81), 0 = else
    dst_data = np.where(tmp_1km == 81, 1, 0).astype(np.uint8)

# write the 1 km pasture mask
with rasterio.open(dst_path, "w", **dst_meta) as dst:
    dst.write(dst_data, 1)

print("Finished:", dst_path)


Finished: mrlc_pasture_2000_1km.tif
